In [110]:
# !python reset_redis.py all

In [111]:
# !python reset_redis.py 'resonator_spectroscopy'
# !python reset_redis.py 'qubit_01_spectroscopy_pulsed'
# !python reset_redis.py 'rabi_oscillations'
# !python reset_redis.py 'motzoi_parameter'
# !python reset_redis.py 'n_rabi_oscillations'
!python reset_redis.py 'resonator_spectroscopy_1'
!python reset_redis.py 'qubit_12_spectroscopy_pulsed'
!python reset_redis.py 'rabi_oscillations_12'
!python reset_redis.py 'ramsey_correction_12'
# !python reset_redis.py 'resonator_spectroscopy_2'
# !python reset_redis.py 'cz_chevron'
# !python reset_redis.py 'cz_calibration'

 remove_node = 'resonator_spectroscopy_1'
 remove_node = 'qubit_12_spectroscopy_pulsed'
 remove_node = 'rabi_oscillations_12'
 remove_node = 'ramsey_correction_12'


In [112]:
!python workers/calibration_supervisor.py

2023-12-04 11:17:08,013 ◆ measurement_utils.py ◆ Starting coupler spectroscopy
Figure(1000x600)
2023-12-04 11:17:09,333 ◆ calibration_supervisor.py ◆ Starting System Calibration
╔════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════╗
║ Qubits: 5                                                                                                                                      ║
║↪ resonator_spectroscopy                                                                                                                        ║
║           ↪ qubit_01_spectroscopy_pulsed                                                                                                       ║
║                         ↪ rabi_oscillations                                                                                                    ║
║                                 ↪ ramsey_correction                                  

In [84]:
import numpy as np
import matplotlib.pyplot as plt
from xarray import load_dataset
from workers.post_processing_worker import Multiplexed_Analysis
# dp = r"data_directory/20231106/20231106-155442-755-1ff0de-cz_chevron/dataset.hdf5"
dp = r'data_directory/20231204/20231204-103444-082-c42866-ramsey_correction_12/dataset.hdf5'
ds = load_dataset(dp)

In [96]:
ds.attrs['name'] = 'new'

In [103]:
ds.attrs['name']

'new'

In [82]:
import redis
redis_connection = redis.Redis(decode_responses=True)
qubits = [ 'q11','q12','q13','q14','q15']
qubit_freq_01_list,qubit_freq_ro_list,qubit_freq_12_list = {},{},{}
for qubit in qubits: 
    print(f"Qubit parameters {qubit}:")
    redis_config = redis_connection.hgetall(f"transmons:{qubit}")
    # Print the redis config line by line
    # for key, value in redis_config.items():
    #     print(f"{key}: {value}")
    print(redis_config['ro_freq'])
    print(redis_config['freq_01'])
    print(redis_config['freq_12'])
    qubit_freq_ro_list[qubit] = redis_config['ro_freq']
    qubit_freq_01_list[qubit] = redis_config['freq_01']
    qubit_freq_12_list[qubit] = redis_config['freq_12']
    print(float(redis_config['freq_12'])-float(redis_config['freq_01']))

Qubit parameters q11:
6934549464.207827
3714915903.6318045
3480295760.3939333
-234620143.23787117
Qubit parameters q12:
6606623788.079234
3366620825.3904705
3173657274.4301305
-192963550.96034002
Qubit parameters q13:
6688273187.131241
3584973087.6373024
3341684358.295266
-243288729.34203625
Qubit parameters q14:
6331337336.993747
3334078482.4869795
3133335946.2948923
-200742536.19208717
Qubit parameters q15:
6933646229.459053
3872830261.1279626
3631887508.935382
-240942752.1925807


In [ ]:
from config_files.VNA_values import VNA_resonator_frequencies, VNA_qubit_frequencies, VNA_f12_frequencies
for qubit in qubits:
    VNA_resonator_frequencies[qubit] = float(qubit_freq_ro_list[qubit])
    VNA_qubit_frequencies[qubit] = float(qubit_freq_01_list[qubit])
    VNA_f12_frequencies[qubit] = float(qubit_freq_12_list[qubit])
# rewrite the VNA_values.py file
with open('config_files/VNA_values.py', 'w') as f:
    f.write(f"VNA_resonator_frequencies = {VNA_resonator_frequencies}\n")
    f.write(f"VNA_f01_frequencies = {VNA_qubit_frequencies}\n")
    f.write(f"VNA_f12_frequencies = {VNA_f12_frequencies}\n")

In [79]:
import numpy as np
print(VNA_resonator_frequencies)
print(VNA_qubit_frequencies)
print(VNA_f12_frequencies)
anahar = (np.array(list(VNA_f12_frequencies.values()))-np.array(list(VNA_qubit_frequencies.values()))) / 1e6
print(anahar)

{'q11': 6934549464.207827, 'q12': 6606623788.079234, 'q13': 6688273187.131241, 'q14': 6331337336.993747, 'q15': 6933646229.459053, 'q16': 6491000000.0, 'q17': 7059000000.0, 'q18': 6712000000.0, 'q19': 6818000000.0, 'q20': 6494000000.0, 'q21': 6751000000.0, 'q22': 6477000000.0, 'q23': 7052000000.0, 'q24': 6583000000.0, 'q25': 6853000000.0}
{'q11': 3714915903.6318045, 'q12': 3366620825.3904705, 'q13': 3584973087.6373024, 'q14': 3334078482.4869795, 'q15': 3872830261.1279626, 'q16': 3189000000.0, 'q17': 3932000000.0, 'q18': 3254000000.0, 'q19': 3921000000.0, 'q20': 3333000000.0, 'q21': 3773000000.0, 'q22': 3326000000.0, 'q23': 3911000000.0, 'q24': 3271000000.0, 'q25': 4009000000.0}
{'q11': 3714925547.1518755, 'q12': 3366214270.0718584, 'q13': 3585207227.3800335, 'q14': 3334248945.025945, 'q15': 3872630397.1224184, 'q16': 2993000000.0, 'q17': 3699000000.0, 'q18': 3154000000.0, 'q19': 3682000000.0, 'q20': 3233000000.0, 'q21': 3542000000.0, 'q22': 3131000000.0, 'q23': 3674000000.0, 'q24': 317

In [ ]:
VNA_resonator_frequencies = {'q11': 6934516975.164495, 'q12': 6606536408.043257, 'q13': 6688260738.410758, 'q14': 6331293538.718747, 'q15': 6933733842.658668, 'q16': 6491000000.0, 'q17': 7059000000.0, 'q18': 6712000000.0, 'q19': 6818000000.0, 'q20': 6494000000.0, 'q21': 6751000000.0, 'q22': 6477000000.0, 'q23': 7052000000.0, 'q24': 6583000000.0, 'q25': 6853000000.0}
VNA_qubit_frequencies = {'q11': 3714911395.272988, 'q12': 3366876737.283394, 'q13': 3584949401.0760307, 'q14': 3334050785.172637, 'q15': 3872848429.703217, 'q16': 3189000000.0, 'q17': 3932000000.0, 'q18': 3254000000.0, 'q19': 3921000000.0, 'q20': 3333000000.0, 'q21': 3773000000.0, 'q22': 3326000000.0, 'q23': 3911000000.0, 'q24': 3271000000.0, 'q25': 4009000000.0}
VNA_f12_frequencies = {'q11': 3480219551.7001905, 'q12': 3173925442.741397, 'q13': 3343692876.3960423, 'q14': 3133039498.0560727, 'q15': 3631777894.9530077, 'q16': 2993000000.0, 'q17': 3699000000.0, 'q18': 3154000000.0, 'q19': 3682000000.0, 'q20': 3233000000.0, 'q21': 3542000000.0, 'q22': 3131000000.0, 'q23': 3674000000.0, 'q24': 3171000000.0, 'q25': 3770000000.0}
